In [1]:
import os
import sys
sys.path.append(os.path.join(os.path.dirname(""),".."))

import custom
import numpy
import pickle
import torch
import torch.nn as nn
from torch.utils.data import DataLoader

In [2]:
with open("ptb.train.txt") as f:
    text = f.readlines()
word_dict, number_dict = custom.make_dict(text)
print(len(word_dict))

9616


In [3]:
corpus = custom.word_num_encoding(text, word_dict)
print(corpus)
print(corpus.shape)

[[   2    3    4 ...    0    0    0]
 [  28   29   30 ...    0    0    0]
 [  42   29   43 ...    0    0    0]
 ...
 [4322 4919   38 ...    0    0    0]
 [  81  831   33 ...    0    0    0]
 [ 112 3581 3582 ...    0    0    0]]
(42068, 98)


In [4]:
word_pair = custom.make_word_pair(corpus, window_size=2)
print(word_pair)
print(word_pair.shape)

[[  2   3]
 [  2   4]
 [  3   2]
 ...
 [ 42 260]
 [ 29  42]
 [ 29  67]]
(3295980, 2)


In [5]:
device = "cuda" if torch.cuda.is_available() else "cpu"

tensor_x = torch.tensor(word_pair[:,0], dtype = torch.long, device = device)
tensor_t = torch.tensor(word_pair[:,1], dtype = torch.long, device = device)
zip_list = list(zip(tensor_x, tensor_t))
dataloader = DataLoader(zip_list,batch_size=100000,shuffle=True)

tensor_x = None
tensor_t = None
zip_list = None

In [ ]:


class NN(nn.Module) :
    def __init__(self, word_size) :
        super().__init__();
        self.f = nn.Embedding(word_size, 100, max_norm=1, padding_idx=0)
        self.g = nn.Linear(100, word_size)
    def forward(self, x) :
        y = self.f(x)
        y = self.g(y)
        return y

F = NN(len(word_dict))
F = F.to(device)
loss_function = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(F.parameters())
epoch = 100

for e in range(epoch) :
    loss_sum = 0
    for x, t in dataloader :
        y = F(t)
    
        loss = loss_function(y, t)
        loss_sum += loss
        
        loss.backward()
        optimizer.step()
        optimizer.zero_grad()
    loss_sum /= len(dataloader)
    
    if (e+1) % 1 == 0 :
        print("epoch {} | loss {}".format(e+1, loss_sum))


epoch 1 | loss 8.662688255310059
epoch 2 | loss 8.619721412658691
epoch 3 | loss 8.577051162719727
epoch 4 | loss 8.53458309173584
epoch 5 | loss 8.492337226867676
epoch 6 | loss 8.450299263000488
epoch 7 | loss 8.408536911010742
epoch 8 | loss 8.367045402526855
epoch 9 | loss 8.32580280303955
epoch 10 | loss 8.284808158874512
epoch 11 | loss 8.24405574798584
epoch 12 | loss 8.203537940979004


In [ ]:
F.to("cpu")
word_vecs = F.state_dict()['f.weight'].numpy()

print(word_vecs.shape)
print(word_vecs)

In [ ]:
word_vecs = word_vecs.tolist()
vector_dict = {}
words = list(word_dict.keys())

for i in (len(word_vecs))
    vector_dict[words[i]] = word_vecs[i]

print(vector_dict)

In [ ]:
with open("word2vec_vector.pkl", mode = "wb") as f :
    pickle.dump(word_vecs, f)